In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Root dataset folder
dataset_root = "MLPR_Dataset_Copy"
output_root = "Processed_Images"

# Ensure output root exists
os.makedirs(output_root, exist_ok=True)

# Define green color range (HSV)
lower_green = np.array([30, 100, 100])
upper_green = np.array([90, 255, 255])

# Recursive function to process images in all subfolders
def process_images_in_folder(folder_path, output_folder):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        output_item_path = os.path.join(output_folder, item)

        if os.path.isdir(item_path):
            # Recursively process subfolders
            process_images_in_folder(item_path, output_item_path)
        elif item.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Read image
            image = cv2.imread(item_path)
            if image is None:
                print(f"Error loading image: {item_path}")
                continue

            # Convert to HSV and create mask
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv, lower_green, upper_green)

            # Morphological operations to refine mask
            kernel = np.ones((7, 7), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

            # Find contours
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contour_debug = image.copy()
            cv2.drawContours(contour_debug, contours, -1, (255, 0, 0), 2)

            # Detect circular well
            well_contour = None
            max_radius = 0
            for cnt in contours:
                (x, y), radius = cv2.minEnclosingCircle(cnt)
                circularity = (4 * np.pi * cv2.contourArea(cnt)) / (cv2.arcLength(cnt, True) ** 2 + 1e-5)

                if radius > max_radius and radius > 30 and circularity > 0.6:
                    max_radius = radius
                    well_contour = (int(x), int(y), int(radius))

            # Draw detected well
            output = image.copy()
            if well_contour:
                x, y, r = well_contour
                cv2.circle(output, (x, y), r, (0, 0, 255), 5)
                cv2.circle(output, (x, y), 5, (255, 0, 0), -1)
                print(f"[✔] Processed: {item_path} - Well detected at ({x}, {y}) with radius {r}")
            else:
                print(f"[✘] Processed: {item_path} - No well detected.")

            # Save processed image and mask
            cv2.imwrite(os.path.join(output_folder, f"processed_{item}"), output)
            cv2.imwrite(os.path.join(output_folder, f"mask_{item}"), mask)

# Start processing from the root dataset folder
process_images_in_folder(dataset_root, output_root)

print("✅ All images processed and saved in:", output_root)

[✔] Processed: MLPR_Dataset_Copy/100 hr/pH8 Enzymatic/IMG07811.JPG - Well detected at (848, 519) with radius 457
[✔] Processed: MLPR_Dataset_Copy/100 hr/pH8 Enzymatic/IMG07810.JPG - Well detected at (1199, 487) with radius 459
[✔] Processed: MLPR_Dataset_Copy/100 hr/pH8 Enzymatic/IMG07812.JPG - Well detected at (777, 605) with radius 455
[✔] Processed: MLPR_Dataset_Copy/100 hr/pH8 Enzymatic/IMG07813.JPG - Well detected at (794, 519) with radius 455
[✔] Processed: MLPR_Dataset_Copy/100 hr/pH8 Enzymatic/IMG07814.JPG - Well detected at (801, 500) with radius 454
[✔] Processed: MLPR_Dataset_Copy/100 hr/pH8 Enzymatic/IMG07815.JPG - Well detected at (889, 532) with radius 455
[✔] Processed: MLPR_Dataset_Copy/100 hr/pH5 Enzymatic/IMG07523.JPG - Well detected at (883, 536) with radius 453
[✔] Processed: MLPR_Dataset_Copy/100 hr/pH5 Enzymatic/IMG07522.JPG - Well detected at (848, 539) with radius 459
[✔] Processed: MLPR_Dataset_Copy/100 hr/pH5 Enzymatic/IMG07521.JPG - Well detected at (1177, 51